# COVID-19 en Chile
> Total de casos confirmados, fallecidos confirmados, pacientes en UCI por región.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
plt.style.use('ggplot')

In [2]:
#hide
from IPython.display import display, Markdown, display_html, HTML

In [3]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)
today = update_date.strftime('%Y-%m-%d')
today

'2020-05-16'

In [4]:
#hide
date_one_week_ago = (update_date - pd.offsets.Day(7)).strftime('%Y-%m-%d')
date_one_week_ago

'2020-05-09'

In [5]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 16/05/2020.

In [6]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [7]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ today + "-CasosConfirmados-totalRegional.csv"
deaths_raw = pd.read_csv(s, index_col='Region')

In [8]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ date_one_week_ago + "-CasosConfirmados-totalRegional.csv"
deaths_one_week_ago = pd.read_csv(s, index_col='Region')

In [9]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')

In [10]:
#hide
data_table = pd.DataFrame()
data_table["Region"] = casos_raw[today].drop("Total").reset_index()["Region"]
data_table["CC"] = casos_raw[today].drop("Total").reset_index()[today]
data_table["CC por 100.000 hab."] = np.round(100000*(casos_raw[today]/tests_raw['Poblacion']).drop('Total'), decimals=1).values
data_table["FC"] = deaths_raw["Fallecidos"].drop("Total").values
data_table["FC por 100.000 hab."] = \
np.round(100000*deaths_raw['Fallecidos'].drop('Total')/(tests_raw['Poblacion'].values), decimals=1).values

In [11]:
#hide
tests_raw = tests_raw.fillna(0)

In [12]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [13]:
#hide
tests_raw.loc['Total'] = tests_total

In [14]:
#hide
tests_table = pd.DataFrame()
tests_table["Región"] = tests_raw.reset_index()['Region']
tests_table["Exámenes informados"] = tests_raw.drop(columns='Poblacion').sum(axis=1).values
tests_table["Exámenes informados por 100.000 habs."] = \
np.round(100000*tests_raw.drop(columns='Poblacion').sum(axis=1).values/tests_raw['Poblacion'], decimals=1).values

In [15]:
#hide
total_confirmed = casos_raw.iloc[-1,-1]
total_deaths = deaths_raw.loc["Total", "Fallecidos"]

In [16]:
#hide
total_confirmed_one_week_ago = casos_raw.iloc[-1,-8]
total_deaths_one_week_ago = deaths_one_week_ago.loc["Total", "Fallecidos"]

In [17]:
#hide
diff_cases = total_confirmed-total_confirmed_one_week_ago
diff_deaths = total_deaths-total_deaths_one_week_ago

In [18]:
#hide
total_confirmed = '{:,}'.format(total_confirmed).replace(',', '.')
total_deaths = '{:,}'.format(total_deaths).replace(',', '.')

In [19]:
#hide
diff_cases = '{:,}'.format(diff_cases).replace(',', '.')
diff_deaths = '{:,}'.format(diff_deaths).replace(',', '.')

# Estadísticas para Chile

In [20]:
#hide_input
HTML(f'<p style="color:#3361ff;"><span style="font-weight:bold;">Casos confirmados:</span> {total_confirmed} (+{diff_cases} en una semana)</p><p style="color:#FF3F3F;"><span style="font-weight:bold;">Fallecimientos confirmados:</span> {total_deaths} (+{diff_deaths} en una semana)</p>')

In [21]:
#hide_input
(data_table.style.set_properties(**{'text-align': 'right'}).background_gradient(cmap='Reds').hide_index()).format({'CC por 100.000 hab.': "{:.1f}", 'FC por 100.000 hab.': "{:.1f}"}).set_caption('Estadísticas por región: Casos confirmados (CC) y fallecidos confirmados (FC)')

Region,CC,CC por 100.000 hab.,FC,FC por 100.000 hab.
Arica y Parinacota,377,206.5,7,2.8
Tarapacá,845,156.5,2,0.5
Antofagasta,1429,149.5,12,1.7
Atacama,146,46.4,0,0.0
Coquimbo,187,7.5,2,0.2
Valparaíso,1365,64.6,28,1.4
Metropolitana,30794,22.4,246,3.0
O’Higgins,329,72.2,5,0.5
Maule,551,54.0,14,1.2
Ñuble,921,533.7,20,3.9


In [22]:
#hide_input
(tests_table[:-1].style.set_properties(**{'text-align': 'right'}).background_gradient(cmap='Blues').hide_index()).format({'Exámenes informados por 100.000 habs.': "{:.1f}"}).set_caption('Estadísticas por región: Exámenes informados desde el 9 de abril')

Región,Exámenes informados,Exámenes informados por 100.000 habs.
Arica y Parinacota,3180,1261.4
Tarapacá,3333,870.8
Antofagasta,7451,1077.0
Atacama,2962,941.2
Coquimbo,2274,272.0
Valparaíso,12149,619.8
Metropolitana,190939,2350.0
O’Higgins,3336,336.6
Maule,10350,914.4
Ñuble,4805,939.3


Nota: Los exámenes informados por región empezaron a informarse sólo desde el 9 de abril.

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de casos confirmados diarios

In [23]:
#hide
# Window size cases
WS_cases = 7

In [24]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [25]:
#hide
aux1 = casos_raw.loc["Total"]
aux2 = [t - s for s, t in zip(aux1, aux1.loc['2020-03-04':])]

In [26]:
#hide
new_cases = pd.DataFrame()
new_cases["Dates"] = casos_raw.loc["Total"].reset_index()["index"].iloc[1:].values
new_cases["New_cases"] = aux2
new_cases["Rolling"] = new_cases["New_cases"].rolling(window=WS_cases).mean()
new_cases["Casos confirmados diarios"] = len(new_cases) * ["Casos confirmados diarios"]
new_cases["Promedio móvil últimos 7 días"] = len(new_cases) * ["Promedio móvil últimos 7 días"]

In [27]:
#hide_input
bars = alt.Chart(new_cases).mark_bar(opacity=0.7).encode(
    x = alt.X('Dates:N', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y = alt.Y('New_cases:Q', axis=alt.Axis(title='Casos confirmados diarios')),
    tooltip = ['Dates', 'New_cases'],
    opacity=alt.Opacity('Casos confirmados diarios', legend=alt.Legend(title=""))
)

line = alt.Chart(new_cases).mark_line(point={
      "filled": True,
      "fill": "firebrick"
    }, color='firebrick').encode(
    x=alt.X('Dates:N', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y = alt.Y('Rolling:Q'),
    shape=alt.Shape('Promedio móvil últimos 7 días', legend=alt.Legend(title=""))
)

(bars + line).properties(
    title='Covid-19 en Chile: Casos confirmados diarios',
    width=600
)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de casos confirmados diarios por región

In [28]:
#hide
# Window size cases
WS_cases = 7

In [29]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo_T.csv",
    index_col='Region')

In [30]:
#hide
data_raw = data_raw.diff().fillna(0).T

In [31]:
#hide
data_raw = data_raw.drop("Total")

In [32]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [33]:
#hide
regiones = data_raw.reset_index()['index'].values
data = data.T
data = data.rename(columns={0: "date", 1: "region", 2: "casos", 3: "codigo region"})

In [34]:
#hide
data["casos"] = data["casos"].astype(int)

In [35]:
#hide_input
input_dropdown = alt.binding_select(options=data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('casos', axis=alt.Axis(title='Casos confirmados')),
    color=color,
    tooltip=['date', 'region', 'casos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
).properties(
    title='Covid-19 en Chile: Casos confirmados diarios por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

line = alt.Chart(data).mark_line(point={
      "filled": True,
      "fill": "firebrick"
    }, color='firebrick').encode(
    x=alt.X('date:N', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y = alt.Y('Rolling:Q'),
    shape=alt.Shape('Promedio móvil últimos 7 días', legend=alt.Legend(title=""))
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de fallecimientos confirmados diarios

In [36]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [37]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [38]:
#hide
for i in np.arange(total_days+1):
  date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
  s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
  data_by_date = pd.read_csv(s)
  if 'Fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Fallecidos"].values
  elif 'Casos fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Casos fallecidos"].values
  else:
    data[date] = data_by_date[" Casos fallecidos"].values

In [39]:
#hide
aux1 = data.loc["Total"]
aux2 = [t - s for s, t in zip(aux1, aux1.loc['2020-03-25':])]

In [40]:
#hide
WS_deaths = 7

In [41]:
#hide
new_deaths = pd.DataFrame()
new_deaths["Dates"] = data.loc["Total"].reset_index()["index"].iloc[1:].values
new_deaths["New_deaths"] = aux2
new_deaths["Rolling"] = new_deaths["New_deaths"].rolling(window=WS_deaths).mean()
new_deaths["Fallecimientos confirmados diarios"] = len(new_deaths) * ["Fallecimientos confirmados diarios"]
new_deaths["Promedio móvil últimos 7 días"] = len(new_deaths) * ["Promedio móvil últimos 7 días"]

In [42]:
#hide_input
bars = alt.Chart(new_deaths).mark_bar(opacity=0.7, color='firebrick').encode(
    x = alt.X('Dates:N', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y = alt.Y('New_deaths:Q', axis=alt.Axis(title='Fallecimientos confirmados diarios')),
    tooltip = ['Dates', 'New_deaths'],
    opacity=alt.Opacity('Fallecimientos confirmados diarios', legend=alt.Legend(title=""))
)

line = alt.Chart(new_deaths).mark_line(point={
      "filled": True,
      "fill": "firebrick"
    }, color='firebrick').encode(
    x=alt.X('Dates:N', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y = alt.Y('Rolling:Q'),
    shape=alt.Shape('Promedio móvil últimos 7 días', legend=alt.Legend(title=""))
)

(bars + line).properties(
    title='Covid-19 en Chile: Fallecimientos confirmados diarios',
    width=600
)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de fallecimientos confirmados diarios por región

In [43]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [44]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [45]:
#hide
for i in np.arange(total_days+1):
  date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
  s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
  data_by_date = pd.read_csv(s)
  if 'Fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Fallecidos"].values
  elif 'Casos fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Casos fallecidos"].values
  else:
    data[date] = data_by_date[" Casos fallecidos"].values

In [46]:
#hide
data = data.T.diff().fillna(0).T

In [47]:
#hide
data = data.drop("Total")

In [48]:
#hide
new_data = pd.DataFrame()
i = 0
for date in data.keys():
    for n, region in enumerate(data.index):
        new_data[i] = date, region, data[date].loc[region], n
        i += 1

In [49]:
#hide
data = data.reset_index()
regiones = data['Region'].values
new_data = new_data.T
new_data = new_data.rename(columns={0: "date", 1: "region", 2: "fallecidos", 3: "codigo region"})

In [50]:
#hide
new_data["fallecidos"] = new_data["fallecidos"].astype(int)

In [51]:
#hide_input

input_dropdown = alt.binding_select(options=new_data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(new_data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecimientos confirmados')),
    color=color,
    tooltip=['date', 'region', 'fallecidos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title='Covid-19 en Chile: Fallecimientos confirmados diarios por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(new_data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de casos confirmados por región

In [52]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [53]:
#hide
data_raw = data_raw.drop("Total")

In [54]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [55]:
#hide
data_raw = data_raw.reset_index()
regiones = data_raw['Region'].values
data = data.T
data = data.rename(columns={0: "date", 1: "region", 2: "casos", 3: "codigo region"})

In [56]:
#hide
data["casos"] = data["casos"].astype(int)

In [57]:
#hide_input
input_dropdown = alt.binding_select(options=data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('casos', axis=alt.Axis(title='Casos confirmados')),
    color=color,
    tooltip=['date', 'region', 'casos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
).properties(
    title='COVID-19 en Chile: Total de casos confirmados por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

Inspirado del [tweet](https://twitter.com/NachoToledoR/status/1244631395781218306) de [@NachoToledoR](https://twitter.com/NachoToledoR).

# Evolución de fallecidos confirmados por región

In [58]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [59]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [60]:
#hide
for i in np.arange(total_days+1):
  date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
  s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
  data_by_date = pd.read_csv(s)
  if 'Fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Fallecidos"].values
  elif 'Casos fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Casos fallecidos"].values
  else:
    data[date] = data_by_date[" Casos fallecidos"].values

In [61]:
#hide
data = data.drop("Total")

In [62]:
#hide
new_data = pd.DataFrame()
i = 0
for date in data.keys():
    for n, region in enumerate(data.index):
        new_data[i] = date, region, data[date].loc[region], n
        i += 1

In [63]:
#hide
data = data.reset_index()
regiones = data['Region'].values
new_data = new_data.T
new_data = new_data.rename(columns={0: "date", 1: "region", 2: "fallecidos", 3: "codigo region"})

In [64]:
#hide
new_data["fallecidos"] = new_data["fallecidos"].astype(int)

In [65]:
#hide_input

input_dropdown = alt.binding_select(options=new_data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(new_data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecidos confirmados')),
    color=color,
    tooltip=['date', 'region', 'fallecidos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title='COVID-19 en Chile: Total de fallecidos confirmados por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(new_data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de pacientes en UCI por región

In [66]:
#hide
data_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto8/UCI.csv',
    index_col='Region')
data_raw = data_raw.drop(columns='Codigo region')
data_raw.loc['Total'] = data_raw.sum(axis=0)
data_raw = data_raw.drop(columns='Poblacion')

In [67]:
#hide
data_raw = data_raw.drop("Total")

In [68]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [69]:
#hide
data_raw = data_raw.reset_index()

In [70]:
#hide
regiones = data_raw['Region'].values

In [71]:
#hide
data = data.T

In [72]:
#hide
data = data.rename(columns={0: "date", 1: "region", 2: "pacientes", 3: "codigo region"})

In [73]:
#hide
data["pacientes"] = data["pacientes"].astype(int)

In [74]:
#hide_input
input_dropdown = alt.binding_select(options=data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('pacientes'),
    color=color,
    tooltip=['date', 'region', 'pacientes'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title=f'COVID-19 en Chile: Pacientes en UCI por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(new_data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(width=600, height=400) | legend

alt.HConcatChart(...)

In [75]:
#hide_input
selection2 = alt.selection_multi(fields=['region'], on='mouseover')

color = alt.condition(selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_area().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('pacientes', stack="normalize"),
    color=color,
    tooltip=['date', 'region', 'pacientes'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title=f'COVID-19 en Chile: Pacientes en UCI por región normalizado'
).add_selection(
    selection2
)

legend = alt.Chart(new_data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection2
)

bars.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

In [76]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [77]:
#hide
regiones = data_raw.drop('Total').reset_index()["Region"].values

In [78]:
#hide
data_raw = data_raw.reset_index()[['Region', today]]
data_raw = data_raw.rename(columns={today: "Casos confirmados"})

In [79]:
#hide
today_es_format = update_date.strftime('%d/%m/%Y')

In [80]:
#hide
#Remove total
data_raw = data_raw.set_index('Region').drop("Total").reset_index()

In [81]:
#hide
bars = alt.Chart(data_raw).mark_bar().encode(
    x = alt.X("Casos confirmados", axis=alt.Axis(title='Casos confirmados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = "Casos confirmados" 
).properties(
    title=f'COVID-19 en Chile: Casos confirmados por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text="Casos confirmados"
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

# Casos confirmados por cada 100.000 habitantes por región

In [82]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')

In [83]:
#hide
data_raw["Poblacion"] = tests_raw["Poblacion"].values
data_raw["Tasa de casos confirmados"] = 100000*data_raw["Casos confirmados"]/tests_raw["Poblacion"].values

In [84]:
#hide_input

bars = alt.Chart(data_raw).mark_bar(opacity=0.8).encode(
    x = alt.X('Tasa de casos confirmados:Q', axis=alt.Axis(title='Casos confirmados por cada 100.000 habitantes')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = ['Region', 'Casos confirmados', 'Poblacion', 'Tasa de casos confirmados'],
    color = alt.Color('Region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
).properties(
    title=f'COVID-19 en Chile: Casos confirmados por cada 100.000 habitantes por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Tasa de casos confirmados:Q', format=',.2f')
)

(bars + text).properties(width=600, height=alt.Step(30))

alt.LayerChart(...)

In [85]:
#hide
cases = data_raw

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

In [86]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today +'-CasosConfirmados-totalRegional.csv'
data_raw = pd.read_csv(s, index_col='Region')

In [87]:
#hide
data_raw = data_raw[data_raw['Fallecidos']>0]
data_raw = data_raw.drop('Total')

In [88]:
#hide
bars = alt.Chart(data_raw.reset_index()).mark_bar().encode(
    x = alt.X('Fallecidos:Q', axis=alt.Axis(title='Fallecidos confirmados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'Fallecidos'
).properties(
    title=f'COVID-19 en Chile: Fallecidos confirmados por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Fallecidos:Q'
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=alt.Step(30))

alt.LayerChart(...)

# Fallecidos confirmados por cada 100.000 habitantes por región

In [89]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today + '-CasosConfirmados-totalRegional.csv'
data_raw = pd.read_csv(s, index_col='Region')

In [90]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')
tests_raw = tests_raw.fillna(0)

In [91]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [92]:
#hide
tests_raw.loc['Total'] = tests_total

In [93]:
#hide
death_rate = 100000*data_raw['Fallecidos']/tests_raw['Poblacion'].values
death_rate = death_rate.sort_values(ascending=True)
death_rate = death_rate[death_rate>0]

In [94]:
#hide
death_rate = pd.DataFrame()
death_rate['Region'] = data_raw.reset_index()['Region']
death_rate['Fallecidos'] = data_raw['Fallecidos'].values
death_rate['Poblacion'] = tests_raw['Poblacion'].values
death_rate['Tasa de fallecimiento'] = 100000*data_raw['Fallecidos'].values/tests_raw['Poblacion'].values
death_rate = death_rate.set_index('Region')

In [95]:
#hide
death_rate = death_rate[death_rate["Fallecidos"]>0]

In [96]:
#hide_input
bars = alt.Chart(death_rate.reset_index()).mark_bar(opacity=0.8).encode(
    x = alt.X('Tasa de fallecimiento:Q', axis=alt.Axis(title='Fallecidos confirmados por cada 100.000 habitantes')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = ['Region', 'Fallecidos', 'Poblacion', 'Tasa de fallecimiento'],
    color = alt.Color('Region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
).properties(
    title=f'COVID-19 en Chile: Fallecidos confirmados por cada 100.000 habitantes por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Tasa de fallecimiento:Q', format=',.2f')
)

(bars + text).properties(width=600, height=alt.Step(30))

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Tasa de letalidad por región
> Tasa de letalidad = fallecidos confirmados / casos confirmados

In [97]:
#hide
CFR = cases.drop(columns="Tasa de casos confirmados")[["Region", "Poblacion", "Casos confirmados"]]
CFR["Fallecidos confirmados"] = data_raw.drop("Total")['Fallecidos'].values
CFR["Tasa de letalidad"] = CFR['Fallecidos confirmados']/CFR['Casos confirmados']

In [98]:
#hide
CFR = CFR[CFR["Fallecidos confirmados"]>0]

In [99]:
#hide
CFR["Texto"] = \
[f"{100*CFR['Tasa de letalidad'].loc[i]:.2f}" for i, val in CFR.iterrows()]

In [100]:
#hide_input
bars = alt.Chart(CFR).mark_bar(opacity=0.8).encode(
    x = alt.X('Tasa de letalidad:Q', axis=alt.Axis(format='%', title='Tasa de letalidad')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = ['Region', 'Fallecidos confirmados', 'Casos confirmados', 'Tasa de letalidad'],
    color = alt.Color('Region:N', scale=alt.Scale(scheme='tableau20'), legend=None)
).properties(
    title=f'COVID-19 en Chile: Tasa de letalidad por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto:Q')
)

(bars + text).properties(width=600, height=alt.Step(30))

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

In [101]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-08-CasosConfirmados-totalRegional.csv'
data_20200408 = pd.read_csv(s, index_col='Region')

In [102]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today +'-CasosConfirmados-totalRegional.csv'
data_update_date = pd.read_csv(s, index_col='Region')

In [103]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')
tests_raw = tests_raw.fillna(0)

In [104]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [105]:
#hide
tests_raw.loc['Total'] = tests_total

In [106]:
#hide
data = pd.DataFrame()
data["Fallecidos"] = data_update_date["Fallecidos"]-data_20200408["Fallecidos"]
data["Casos"] = data_update_date["Casos  totales  acumulados"]-data_20200408["Casos  totales"]
data["Tests"] = tests_raw.drop(columns='Poblacion').sum(axis=1).values

In [107]:
#hide
data_per_capita = pd.DataFrame()
data_per_capita["Fallecidos"] = np.round(100000*data["Fallecidos"]/tests_raw["Poblacion"].values, decimals=1)
data_per_capita["Casos"] = np.round(100000*data["Casos"]/tests_raw["Poblacion"].values, decimals=1)
data_per_capita["Tests"] = np.round(100000*data["Tests"]/tests_raw["Poblacion"].values, decimals=1)

In [108]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv", 
    index_col='Region')

In [109]:
#hide
casos_20200408 = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-08-CasosConfirmados-totalRegional.csv', index_col='Region')

In [110]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today + '-CasosConfirmados-totalRegional.csv'
casos_update_date = pd.read_csv(s, index_col='Region')

In [111]:
#hide
positividad = 100*(casos_update_date['Casos  totales  acumulados'] - casos_20200408['Casos  totales'])/tests_raw.drop(columns='Poblacion').sum(axis=1).values

In [112]:
#hide
tests_raw['tests informados'] = tests_raw.drop(columns='Poblacion').sum(axis=1)

In [113]:
#hide
tests_raw['tasa de tests'] = np.round(100000*tests_raw['tests informados']/tests_raw['Poblacion'], decimals=1)

In [114]:
#hide
bars = alt.Chart(tests_raw.drop('Total').reset_index()).mark_bar().encode(
    x = alt.X('tests informados:Q', axis=alt.Axis(title='Exámenes informados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'tests informados'
).properties(
    title='COVID-19 en Chile: Total de exámenes informados por región desde el 9 de abril'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('tests informados:Q', format='.0f')
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

# Exámenes informados por región por cada 100.000 habitantes desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

In [115]:
#hide_input
bars = alt.Chart(tests_raw.drop('Total').reset_index()).mark_bar(opacity=0.8).encode(
    x = alt.X('tasa de tests:Q', axis=alt.Axis(title='Exámenes informados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'tasa de tests',
    color = alt.Color('Region:N', scale=alt.Scale(scheme='tableau20'), legend=None)
).properties(
    title='COVID-19 en Chile: Exámenes informados por región por cada 100.000 habitantes desde el 9 de abril'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('tasa de tests:Q', format='.1f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=alt.Step(30))

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

In [116]:
#hide
data = data.sort_values(by="Tests", ascending=False)
data_per_capita = data_per_capita.sort_values(by="Tests", ascending=False)

In [117]:
#hide
data["codigo"]=np.arange(len(data))
data_per_capita["codigo"]=np.arange(len(data))

In [118]:
#hide
bar = alt.Chart(data.drop("Total").reset_index()).mark_bar(size=25, opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados')),
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

tick0 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=30,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

tick1 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=30,  # controls width of tick.
).encode(
    x='Casos:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

tick2 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=30,  # controls width of tick.
).encode(
    x='Fallecidos:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

(bar + tick0 + tick1 + tick2).properties(
    title='COVID-19 en Chile: Exámenes informados, casos confirmados y fallecidos confirmados por región',
    width=700,
    height=alt.Step(30)
)

alt.LayerChart(...)

# Exámenes, casos confirmados, y fallecimientos confirmados por región por cada 100.000 habitantes desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por región por cada 100.000 habitantes desde el 9 de abril

In [119]:
#hide_input
bar = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_bar(size=25, opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados')),
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

tick0 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=25,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

tick1 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=25,  # controls width of tick.
).encode(
    x='Casos:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

tick2 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=25,  # controls width of tick.
).encode(
    x='Fallecidos:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos']
)

(bar + tick0 + tick1 + tick2).properties(
    title='COVID-19 en Chile: Exámenes informados, casos confirmados y fallecidos confirmados por región por cada 100.000 habitantes',
    height=alt.Step(30),
    width=700)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de la OCDE por cada 1 millón de habitantes

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

In [120]:
#hide
test_data_raw = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv')

In [121]:
#hide
list_OCDE = ['Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Colombia',
                  'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
                  'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
                  'Israel', 'Italy', 'Japan', 'Korea', 'Latvia',
                  'Lithuania', 'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand',
                  'Norway', 'Poland', 'Portugal', 'Slovakia', 'Slovenia',
                  'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom']

In [122]:
#hide
test_data = pd.DataFrame()
for country in list_OCDE:
    test_data[country] = test_data_raw[test_data_raw['Entity'].str.contains(country)].iloc[-1]

In [123]:
#hide
# United States tiene dos series. Utilizo la que tiene datos más recientes.
test_data['United States'] = test_data_raw[test_data_raw['Entity'].str.contains('United States - inconsistent units')].iloc[-1]

In [124]:
#hide
test_data = test_data.T

In [125]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['Cumulative total']

In [126]:
#hide
list_OCDE = list_OCDE + ['United States']

In [127]:
#hide
full_data = pd.read_csv("https://covid.ourworldindata.org/data/ecdc/full_data.csv", index_col='date')

In [128]:
#hide
cases = pd.Series(dtype='float64')
deaths = pd.Series(dtype='float64')
for country in list_OCDE:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_cases']
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_deaths']

In [129]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [130]:
#hide
locations_data = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')

In [131]:
#hide
population = pd.Series(dtype='float64')
for country in list_OCDE:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [132]:
#hide
data['Population'] = population

In [133]:
#hide
data['Tests'] = np.round(1000000*data['Tests'].astype(float)/data['Population'])
data['Cases'] = np.round(1000000*data['Cases']/data['Population'])
data['Deaths'] = np.round(1000000*data['Deaths']/data['Population'])

In [134]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [135]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(size=15, opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
).properties(
    width=alt.Step(70)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

(bar + tick0 + tick1 + tick2).properties(
    title='COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes',
    height=alt.Step(16),
    width=700
)

alt.LayerChart(...)

Fuentes: ECDC, [Our World in Data](https://ourworldindata.org/)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de la OCDE por cada 1 millón de habitantes en los últimos 3 días

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

In [136]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['3-day rolling mean daily change']

In [137]:
#hide
cases = pd.Series(dtype='float64')
deaths = pd.Series(dtype='float64')
for country in list_OCDE:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_cases'].mean()
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_deaths'].mean()

In [138]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [139]:
#hide
population = pd.Series(dtype='float64')
for country in list_OCDE:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [140]:
#hide
data['Population'] = population

In [141]:
#hide
data['Tests'] = np.round(1000000*data['Tests'].astype(float)/data['Population'], decimals=1)
data['Cases'] = np.round(1000000*data['Cases']/data['Population'], decimals=1)
data['Deaths'] = np.round(1000000*data['Deaths']/data['Population'], decimals=1)

In [142]:
#hide
data = data.dropna()

In [143]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [144]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(size=15, opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title=f'Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes\n(promedio de los últimos 3 días)')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

(bar + tick0 + tick1 + tick2).properties(
    title=f'COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes\n(promedio de los últimos 3 días)',
    height=alt.Step(16),
    width=700
)

alt.LayerChart(...)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de latinoamérica por cada 1 millón de habitantes

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

Nota: Muchos países de latinoamérica no tienen datos disponibles.

In [145]:
#hide
list_latinoamerica = ['Argentina', 'Bolivia', 'Chile', 'Colombia',
                     'Costa Rica', 'Cuba', 'Ecuador', 'El Salvador',
                     'Mexico', 'Panama', 'Paraguay',
                     'Peru', 'Uruguay']

In [146]:
#hide
test_data = pd.DataFrame()
for country in list_latinoamerica:
    test_data[country] = test_data_raw[test_data_raw['Entity'].str.contains(country)].iloc[-1]

In [147]:
#hide
test_data = test_data.T

In [148]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['Cumulative total']

In [149]:
#hide
cases = pd.Series(dtype='float64')
deaths = pd.Series(dtype='float64')
for country in list_latinoamerica:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_cases']
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_deaths']

In [150]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [151]:
#hide
population = pd.Series(dtype='float64')
for country in list_latinoamerica:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [152]:
#hide
data['Population'] = population

In [153]:
#hide
data['Tests'] = np.round(1000000*data['Tests'].astype(float)/data['Population'])
data['Cases'] = np.round(1000000*data['Cases']/data['Population'])
data['Deaths'] = np.round(1000000*data['Deaths']/data['Population'])

In [154]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [155]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(size=15, opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

(bar + tick0 + tick1 + tick2).properties(
    title='COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes',
    height=alt.Step(16),
    width=700  # controls width of bar.
)

alt.LayerChart(...)

Fuentes: ECDC, [Our World in Data](https://ourworldindata.org/)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de latinoamérica por cada 1 millón de habitantes en los últimos 3 días

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

Nota: Muchos países de latinoamérica no tienen datos disponibles.

In [156]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['3-day rolling mean daily change']

In [157]:
#hide
cases = pd.Series(dtype='float64')
deaths = pd.Series(dtype='float64')
for country in list_latinoamerica:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_cases'].mean()
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_deaths'].mean()

In [158]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [159]:
#hide
population = pd.Series(dtype='float64')
for country in list_latinoamerica:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [160]:
#hide
data['Population'] = population

In [161]:
#hide
data['Tests'] = np.round(1000000*data['Tests'].astype(float)/data['Population'], decimals=1)
data['Cases'] = np.round(1000000*data['Cases']/data['Population'], decimals=1)
data['Deaths'] = np.round(1000000*data['Deaths']/data['Population'], decimals=1)

In [162]:
#hide
data = data.dropna()

In [163]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [164]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(size=15, opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title=f'Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes\n(promedio de los últimos 3 días)')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=15,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

(bar + tick0 + tick1 + tick2).properties(
    title=f'COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes\n(promedio de los últimos 3 días)',
    height=alt.Step(16),
    width=700  # controls width of bar.
)

alt.LayerChart(...)